<a href="https://www.kaggle.com/code/trcnguyncng/dif-sr?scriptVersionId=241587299" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/configs/configs/yelp.yaml
/kaggle/input/configs/configs/Amazon_Toys_and_Games.yaml
/kaggle/input/configs/configs/Amazon_Beauty.yaml
/kaggle/input/configs/configs/Amazon_Sports_and_Outdoors.yaml
/kaggle/input/sequential-recommendation/dataset/Amazon_Beauty/Amazon_Beauty.inter
/kaggle/input/sequential-recommendation/dataset/Amazon_Beauty/Amazon_Beauty.item
/kaggle/input/sequential-recommendation/dataset/Amazon_Toys_and_Games/Amazon_Toys_and_Games.item
/kaggle/input/sequential-recommendation/dataset/Amazon_Toys_and_Games/Amazon_Toys_and_Games.inter
/kaggle/input/sequential-recommendation/dataset/Amazon_Sports_and_Outdoors/Amazon_Sports_and_Outdoors.inter
/kaggle/input/sequential-recommendation/dataset/Amazon_Sports_and_Outdoors/Amazon_Sports_and_Outdoors.item


In [2]:
import pandas as pd

# Đọc file interactions (dùng sep phù hợp: ',' hoặc '\t')
inter = pd.read_csv("/kaggle/input/sequential-recommendation/dataset/Amazon_Beauty/Amazon_Beauty.inter", sep='\t')  # hoặc sep=','

In [3]:
inter.head()

,user_id:token,item_id:token,rating:float,timestamp:float
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [4]:
print("Installing RecBole library...")
!pip install recbole -q
print("RecBole installation complete.")

Installing RecBole library...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bayesian-optimization 2.0.3 requires colorama<0.5.0,>=0.4.6, but you have colorama

In [5]:
# --- Cell 1: Install RecBole and Import Libraries ---
# Ensure Internet & GPU Accelerator are enabled in Notebook settings

# Imports needed for the provided run_recbole function and general setup
import logging
from logging import getLogger
import torch
import pickle
import os
import yaml # To potentially read/modify config if needed

# Import RecBole components specifically required by the run_recbole function
from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

print("Libraries imported.")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

2025-05-24 11:36:42.386412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748086602.576961      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748086602.630842      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Libraries imported.


In [6]:
# --- Cell 2: Define the run_recbole function and its helpers ---
# Using the code you provided

# @Time   : 2020/10/6
# @Author : Shanlei Mu
# @Email  : slmu@ruc.edu.cn

"""
recbole.quick_start
########################
"""

def run_recbole(model=None, dataset=None, config_file_list=None, config_dict=None, saved=True):
    r""" A fast running api, which includes the complete process of
    training and testing a model on a specified dataset

    Args:
        model (str, optional): Model name. Defaults to ``None``.
        dataset (str, optional): Dataset name. Defaults to ``None``.
        config_file_list (list, optional): Config files used to modify experiment parameters. Defaults to ``None``.
        config_dict (dict, optional): Parameters dictionary used to modify experiment parameters. Defaults to ``None``.
        saved (bool, optional): Whether to save the model. Defaults to ``True``.
    """
    # configurations initialization
    # Pass model and dataset *directly* to Config constructor as well
    config = Config(model=model, dataset=dataset, config_file_list=config_file_list, config_dict=config_dict)
    init_seed(config['seed'], config['reproducibility'])
    # logger initialization
    init_logger(config)
    logger = getLogger()

    logger.info(f"Running with config:\n{config}")

    # dataset filtering
    # create_dataset will use the 'dataset' name and 'data_path' from config
    dataset_obj = create_dataset(config)
    if config['save_dataset']:
        dataset_obj.save() # Note: dataset_obj is the correct object name here
    logger.info(dataset_obj)

    # dataset splitting
    # data_preparation also uses the config object
    train_data, valid_data, test_data = data_preparation(config, dataset_obj)
    if config['save_dataloaders']:
        save_split_dataloaders(config, dataloaders=(train_data, valid_data, test_data))

    # model loading and initialization
    # get_model uses the 'model' name from config
    model_instance = get_model(config['model'])(config, train_data.dataset).to(config['device'])
    logger.info(model_instance)

    # trainer loading and initialization
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model_instance) # Pass model_instance here

    # model training
    best_valid_score, best_valid_result = trainer.fit(
        train_data, valid_data, saved=saved, show_progress=config['show_progress']
    )

    # model evaluation
    test_result = trainer.evaluate(test_data, load_best_model=saved, show_progress=config['show_progress'])

    logger.info(set_color('best valid ', 'yellow') + f': {best_valid_result}')
    logger.info(set_color('test result', 'yellow') + f': {test_result}')

    return {
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

# --- You can keep or remove the other functions (objective_function, load_data_and_model) ---
# --- They are not directly needed to run the basic experiment ---

print("run_recbole function defined.")

run_recbole function defined.


In [7]:
# --- Ô 3: Định nghĩa Tham số và Từ điển Cấu hình ---

# --- Định nghĩa Đường dẫn dựa trên thiết lập Kaggle của bạn ---
# Đây là thư mục CHA chứa thư mục con 'Amazon_Beauty'
kaggle_data_base_path = '/kaggle/input/sequential-recommendation/dataset/'

# --- Định nghĩa các Thông tin cụ thể cho Thử nghiệm ---
ten_mo_hinh = 'SASRec'
ten_dataset = 'Amazon_Beauty' # Tên này PHẢI khớp với tên thư mục con trong kaggle_data_base_path
id_gpu = '0' # Sử dụng GPU đầu tiên có sẵn trên Kaggle

# --- Định nghĩa Từ điển Cấu hình ĐẦY ĐỦ ---
# Sử dụng từ điển bạn đã cung cấp, đảm bảo data_path là chính xác
# và thêm phần sửa lỗi cho train_neg_sample_args
final_config_dict = {
    # --- Cấu hình Dataset Quan trọng ---
    'dataset': ten_dataset,       # Cho RecBole biết cần tìm thư mục con nào
    'data_path': kaggle_data_base_path, # Cho RecBole biết thư mục CHA của tất cả các dataset
    'USER_ID_FIELD': 'user_id',    # Tên cột ID người dùng
    'ITEM_ID_FIELD': 'item_id',    # Tên cột ID mặt hàng
    'TIME_FIELD': 'timestamp',     # Tên cột thời gian
    'load_col': {                  # Chỉ định các cột cần tải
        'inter': ['user_id', 'item_id', 'timestamp'], # Tương tác
        'item': ['item_id']           # Mặt hàng (giữ tối thiểu cho SASRec gốc)
                                      # Chỉ thêm các thuộc tính khác nếu mô hình sử dụng chúng
    },
    'ITEM_LIST_LENGTH_FIELD': 'item_length', # Tên trường độ dài chuỗi (mặc định)
    'LIST_SUFFIX': '_list',            # Hậu tố cho các trường chuỗi (mặc định)
    'MAX_ITEM_LIST_LENGTH': 50,        # Độ dài chuỗi tối đa
    'user_inter_num_interval': '[5,inf)', # Lọc người dùng có ít hơn 5 tương tác
    'item_inter_num_interval': '[5,inf)', # Lọc mặt hàng có ít hơn 5 tương tác

    # --- Cấu hình Huấn luyện ---
    'epochs': 30,                  # Số epoch huấn luyện (điều chỉnh nếu cần)
    'train_batch_size': 2048,      # Điều chỉnh dựa trên bộ nhớ GPU Kaggle (T4/P100)
    'eval_batch_size': 512,       # Kích thước batch đánh giá
    'learning_rate': 0.001,        # Tốc độ học
    'stopping_step': 10,           # Số epoch kiên nhẫn chờ cải thiện trước khi dừng sớm (Early Stopping)

    # --- Cấu hình Đánh giá ---
    'eval_setting': 'TO_LS,full',  # Sắp xếp theo TimeStamp, Chia Leave-one-out, Đánh giá full ranking
    'metrics': ['Recall', 'NDCG', 'Hit', 'Precision', 'MRR'], # Các độ đo cần tính
    'topk': [10, 20],              # Các giá trị K cho độ đo top-K
    'valid_metric': 'NDCG@10',     # Độ đo dùng để dừng sớm và lưu mô hình tốt nhất
    'eval_args': {
        'split': {'LS': 'valid_and_test'}, # Chiến lược chia (Leave-one-out)
        'order': 'TO',                  # Thứ tự (TimeStamp)
        'mode': 'full'                  # Chế độ đánh giá (full ranking)
    },

    # --- Cấu hình Mô hình (Cụ thể cho SASRec) ---
    'model': ten_mo_hinh,           # Đảm bảo khớp với mô hình chúng ta muốn chạy
    'embedding_size': 64,           # Kích thước vector embedding
    'hidden_size': 64,              # Kích thước lớp ẩn (giống embedding_size trong SASRec gốc)
    'inner_size': 256,              # Kích thước lớp bên trong của FeedForward Network
    'n_layers': 2,                  # Số lớp Transformer
    'n_heads': 2,                   # Số lượng attention head
    'hidden_dropout_prob': 0.5,     # Tỷ lệ dropout lớp ẩn
    'attn_dropout_prob': 0.5,       # Tỷ lệ dropout trong lớp attention
    'hidden_act': 'gelu',           # Hàm kích hoạt
    'layer_norm_eps': 1e-12,        # Epsilon cho Layer Normalization
    'initializer_range': 0.02,      # Khoảng giá trị khởi tạo trọng số
    'loss_type': 'CE',              # Loại hàm mất mát (Cross Entropy)

    # --- Vô hiệu hóa tường minh việc lấy mẫu phủ định cho hàm mất mát CE ---
    # --- ĐÂY LÀ PHẦN SỬA LỖI ---
    'train_neg_sample_args': None,

    # --- Cấu hình Phần cứng ---
    'gpu_id': id_gpu,               # ID của GPU muốn sử dụng
    'worker': 4,                    # Số tiến trình tải dữ liệu (điều chỉnh dựa trên CPU Kaggle, bắt đầu thấp hơn)
    'reproducibility': True,        # Đảm bảo kết quả có thể tái lập (tùy chọn)
    'seed': 2023,                   # Hạt giống ngẫu nhiên (tùy chọn)
    'state': 'INFO',                # Mức độ log (INFO, DEBUG, WARNING, ERROR)
    'show_progress': True           # Hiển thị thanh tiến trình tqdm
}

# --- Kiểm tra các đường dẫn dataset được ngụ ý từ cấu hình ---
dataset_dir_check = os.path.join(final_config_dict['data_path'], final_config_dict['dataset'])
print(f"Kiểm tra thư mục dataset: {dataset_dir_check}")
if not os.path.isdir(dataset_dir_check):
     print(f"LỖI: Không tìm thấy thư mục dataset dự kiến tại {dataset_dir_check}")
     print("Vui lòng đảm bảo cấu trúc Bộ dữ liệu 'sequential-recommendation' trên Kaggle là chính xác.")
else:
    # Kiểm tra các tệp cụ thể
    inter_file_check = os.path.join(dataset_dir_check, f"{final_config_dict['dataset']}.inter")
    item_file_check = os.path.join(dataset_dir_check, f"{final_config_dict['dataset']}.item")
    files_ok = True
    if not os.path.exists(inter_file_check):
        print(f"LỖI: Không tìm thấy tệp tương tác (.inter) tại {inter_file_check}")
        files_ok = False
    if not os.path.exists(item_file_check):
         print(f"LỖI: Không tìm thấy tệp mặt hàng (.item) tại {item_file_check}")
         files_ok = False
    if files_ok:
        print("Thư mục dataset và các tệp cần thiết (.inter, .item) dường như đã có.")

print("\nTừ điển cấu hình đã được chuẩn bị với train_neg_sample_args=None.")
# Tùy chọn: In ra một phần của từ điển để kiểm tra
# print("\nMột số giá trị cấu hình mẫu:")
# print(f"  Mô hình: {final_config_dict['model']}")
# print(f"  Dataset: {final_config_dict['dataset']}")
# print(f"  Đường dẫn dữ liệu: {final_config_dict['data_path']}")
# print(f"  Số Epochs: {final_config_dict['epochs']}")

Kiểm tra thư mục dataset: /kaggle/input/sequential-recommendation/dataset/Amazon_Beauty
Thư mục dataset và các tệp cần thiết (.inter, .item) dường như đã có.

Từ điển cấu hình đã được chuẩn bị với train_neg_sample_args=None.


In [8]:
# --- Ô 4: Chạy Thử nghiệm SASRec sử dụng config_dict ---

print(f"\nBắt đầu chạy RecBole cho Mô hình: {final_config_dict['model']}, Dataset: {final_config_dict['dataset']}")
print("Sử dụng cấu hình được định nghĩa trực tiếp trong `final_config_dict`.")

# Kiểm tra lại đường dẫn dataset trước khi chạy
dataset_dir_check = os.path.join(final_config_dict['data_path'], final_config_dict['dataset'])
inter_file_check = os.path.join(dataset_dir_check, f"{final_config_dict['dataset']}.inter")
item_file_check = os.path.join(dataset_dir_check, f"{final_config_dict['dataset']}.item")

if os.path.isdir(dataset_dir_check) and os.path.exists(inter_file_check) and os.path.exists(item_file_check):
    print("Các tệp dataset hợp lệ. Bắt đầu chạy run_recbole...")
    try:
        ket_qua = run_recbole(
            # Tên model và dataset chủ yếu dùng để khởi tạo đối tượng Config,
            # các giá trị bên trong final_config_dict sẽ được ưu tiên khi chạy
            model=final_config_dict['model'],
            dataset=final_config_dict['dataset'],
            config_dict=final_config_dict,  # Truyền từ điển cấu hình vào ĐÂY
            config_file_list=None           # Khẳng định không dùng tệp cấu hình ngoài
        )
        print("\n--- Thử nghiệm Hoàn tất ---")
        print("Kết quả:")
        # In kết quả ra dạng dễ đọc hơn
        import json
        print(json.dumps(ket_qua, indent=4, ensure_ascii=False)) # ensure_ascii=False để hiển thị tiếng Việt nếu có

    except Exception as e:
        print(f"\n--- Đã xảy ra lỗi trong quá trình thực thi ---")
        print(e)
        # Có thể thêm log lỗi chi tiết hơn ở đây nếu cần
        import traceback
        traceback.print_exc()
else:
    print("\nBỏ qua thực thi vì không tìm thấy thư mục hoặc tệp dataset dựa trên cấu hình.")


# Kiểm tra thư mục 'saved' được tạo ra
print("\nLiệt kê nội dung thư mục 'saved' (nếu được tạo):")
# Thư mục 'saved' thường được tạo trong thư mục làm việc hiện tại (/kaggle/working/)
if os.path.exists('saved'):
    !ls -lt saved/
else:
    print("Không tìm thấy thư mục 'saved'.")


Bắt đầu chạy RecBole cho Mô hình: SASRec, Dataset: Amazon_Beauty
Sử dụng cấu hình được định nghĩa trực tiếp trong `final_config_dict`.
Các tệp dataset hợp lệ. Bắt đầu chạy run_recbole...


/usr/local/lib/python3.11/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.11/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c


--- Thử nghiệm Hoàn tất ---
Kết quả:
{
    "best_valid_score": 0.0517,
    "valid_score_bigger": true,
    "best_valid_result": {
        "recall@10": 0.1036,
        "recall@20": 0.1438,
        "ndcg@10": 0.0517,
        "ndcg@20": 0.0618,
        "hit@10": 0.1036,
        "hit@20": 0.1438,
        "precision@10": 0.0104,
        "precision@20": 0.0072,
        "mrr@10": 0.0357,
        "mrr@20": 0.0384
    },
    "test_result": {
        "recall@10": 0.0845,
        "recall@20": 0.1198,
        "ndcg@10": 0.0421,
        "ndcg@20": 0.051,
        "hit@10": 0.0845,
        "hit@20": 0.1198,
        "precision@10": 0.0084,
        "precision@20": 0.006,
        "mrr@10": 0.0291,
        "mrr@20": 0.0315
    }
}

Liệt kê nội dung thư mục 'saved' (nếu được tạo):
total 10336
-rw-r--r-- 1 root root 10583970 May 24 11:43 SASRec-May-24-2025_11-37-19.pth


In [9]:
# --- Ô Code Kiểm tra GPU ---

import torch

print("Đang kiểm tra trạng thái GPU với PyTorch...")

# Kiểm tra xem CUDA (nền tảng cho GPU NVIDIA) có khả dụng không
if torch.cuda.is_available():
    print("\n✅ Chúc mừng! PyTorch đã phát hiện và có thể sử dụng GPU.")

    # Lấy số lượng GPU có sẵn
    gpu_count = torch.cuda.device_count()
    print(f"   - Số lượng GPU tìm thấy: {gpu_count}")

    # In thông tin về từng GPU (thường chỉ có 1 hoặc 2 trên Kaggle)
    for i in range(gpu_count):
        print(f"   - Thông tin GPU {i}:")
        print(f"     + Tên: {torch.cuda.get_device_name(i)}")
        # Bạn có thể thêm các thông tin khác nếu muốn, ví dụ:
        # print(f"     + Khả năng tính toán (Compute Capability): {torch.cuda.get_device_capability(i)}")
        # print(f"     + Tổng bộ nhớ: {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GB") # Chia cho 1024^3 để ra GB

    # Lấy chỉ số của GPU hiện tại mà PyTorch đang sử dụng
    current_gpu_index = torch.cuda.current_device()
    print(f"\n   - PyTorch hiện đang sử dụng GPU có chỉ số: {current_gpu_index} ({torch.cuda.get_device_name(current_gpu_index)})")

    print("\n==> Điều này có nghĩa là các tính toán trong RecBole (huấn luyện, đánh giá) SẼ được thực hiện trên GPU, giúp tăng tốc đáng kể.")
    print("==> Hãy đảm bảo rằng bạn đã chọn đúng loại GPU mong muốn trong cài đặt 'Accelerator' của Kaggle.")

else:
    print("\n❌ Thông báo: PyTorch không phát hiện thấy GPU nào tương thích CUDA.")
    print("   - Code sẽ chạy hoàn toàn bằng CPU.")
    print("\n==> Quá trình huấn luyện mô hình Deep Learning có thể sẽ RẤT CHẬM.")
    print("==> Nếu bạn muốn sử dụng GPU:")
    print("    1. Hãy kiểm tra lại cài đặt 'Accelerator' ở thanh công cụ bên phải của Kaggle.")
    print("    2. Chọn một loại GPU (ví dụ: T4 x2 hoặc P100).")
    print("    3. Khởi động lại phiên làm việc (Restart session) khi được yêu cầu.")
    print("    4. Chạy lại ô code kiểm tra này.")

print("\n--- Kiểm tra GPU hoàn tất ---")

Đang kiểm tra trạng thái GPU với PyTorch...

✅ Chúc mừng! PyTorch đã phát hiện và có thể sử dụng GPU.
   - Số lượng GPU tìm thấy: 1
   - Thông tin GPU 0:
     + Tên: Tesla T4

   - PyTorch hiện đang sử dụng GPU có chỉ số: 0 (Tesla T4)

==> Điều này có nghĩa là các tính toán trong RecBole (huấn luyện, đánh giá) SẼ được thực hiện trên GPU, giúp tăng tốc đáng kể.
==> Hãy đảm bảo rằng bạn đã chọn đúng loại GPU mong muốn trong cài đặt 'Accelerator' của Kaggle.

--- Kiểm tra GPU hoàn tất ---
